In [44]:
import pandas as pd
import heapq

df_gare = pd.read_csv('./timetables_clear.csv', sep=';')

graph_gare = {}

def add_to_graph(df_row):
    if type(df_row['depart']) != str or type(df_row['arrivee']) != str:
        print("ERREUR", df_row['depart'], df_row['arrivee'], df_row['duree'])
    else:
        gare_from = df_row['depart'].replace('\'', ' ')
        gare_to = df_row['arrivee'].replace('\'', ' ')
        travel_time = float(df_row['duree'])

        if type(gare_from) != str or type(gare_to) != str:
            print("ERREUR", gare_from, gare_to)

        if gare_from in graph_gare:
            graph_gare[gare_from][gare_to] = travel_time
        else:
            graph_gare[gare_from] = {gare_to:travel_time}
        
        if gare_to in graph_gare:
            graph_gare[gare_to][gare_from] = travel_time
        else:
            graph_gare[gare_to] = {gare_from:travel_time}


df_gare.apply(add_to_graph, axis=1)


print(graph_gare)


def dijkstra(graph, start):
    shortest_paths = {vertex: float('infinity') for vertex in graph}
    shortest_paths[start] = 0
    previous_nodes = {vertex: None for vertex in graph}
    nodes = graph.keys()

    while nodes:
        current_node = None
        for node in nodes:
            if current_node is None:
                current_node = node
            elif shortest_paths[node] < shortest_paths[current_node]:
                current_node = node

        if shortest_paths[current_node] == float('infinity'):
            break

        for neighbour, weight in graph[current_node].items():
            new_path = shortest_paths[current_node] + weight
            if new_path < shortest_paths[neighbour]:
                shortest_paths[neighbour] = new_path
                previous_nodes[neighbour] = current_node

        nodes = {node for node in nodes if node != current_node}

    return previous_nodes, shortest_paths


def dijkstra_with_path(graph, start):
    pq = []  # Priority queue to represent the min-heap
    heapq.heappush(pq, (0, start, None))  # (distance, current_node, predecessor_node)
    shortest_paths = {vertex: float('infinity') for vertex in graph}
    shortest_paths[start] = 0
    predecessors = {vertex: None for vertex in graph}
    visited = set()

    while pq:
        current_distance, current_vertex, predecessor = heapq.heappop(pq)
        
        if current_vertex in visited:
            continue
        visited.add(current_vertex)
        predecessors[current_vertex] = predecessor

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            if distance < shortest_paths[neighbor] and neighbor not in visited:
                shortest_paths[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor, current_vertex))

    return shortest_paths, predecessors


def reconstruct_path(predecessors, start, end):
    current = end
    path = []
    while current != start:
        if current is None:
            return None  # Path not found
        path.append(current)
        current = predecessors[current]
    path.append(start)
    path.reverse()
    return path

graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}
# Usage
start, end = 'Gare de Boulogne Ville', 'Gare de Toulon'
shortest_distances, predecessors = dijkstra_with_path(graph_gare, start)
path = reconstruct_path(predecessors, start, end)

print(f"Shortest path from {start} to {end}: {path}")
print(f"Total distance: {shortest_distances[end]}")



ERREUR Bourges-Gare-Routière nan 65
{'Gare de Le Havre': {'Gare de Paris-St-Lazare': 138.0, 'Gare de Montivilliers': 15.0, 'Gare de Rolleville': 26.0, 'Gare de Fécamp': 46.0, 'Gare de Rouen-Rive-Droite': 61.0}, 'Gare de Paris-St-Lazare': {'Gare de Le Havre': 138.0, 'Gare de Dieppe': 145.0, 'Gare de Rouen-Rive-Droite': 97.0, 'Gare de Cherbourg': 194.0, 'Gare de Caen': 149.0, 'Gare de Trouville-Deauville': 147.0, 'Gare de Vernon-Giverny': 60.0, 'Gare de Oissel': 94.0, 'Gare de Serquigny': 105.0}, 'Gare de Dieppe': {'Gare de Paris-St-Lazare': 145.0, 'Gare de Rouen-Rive-Droite': 63.0}, 'Gare de Rouen-Rive-Droite': {'Gare de Paris-St-Lazare': 97.0, 'Gare de Caen': 100.0, 'Gare de Mantes-la-Jolie': 125.0, 'Gare de Amiens': 186.0, 'Gare de Lille Flandres': 168.0, 'Gare de Le Havre': 61.0, 'Gare de Yvetot': 34.0, 'Gare de Dieppe': 63.0}, 'Gare de Cherbourg': {'Gare de Paris-St-Lazare': 194.0, 'Gare de Caen': 174.0}, 'Gare de Caen': {'Gare de Paris-St-Lazare': 149.0, 'Gare de Le Mans': 111.0, '